In [11]:
from XRDXRFutils import DataSXRF, SpectraSXRF, FluorescenceSXRF
datadir = "/home/rosario/xmimsim/syntetic/ElGreco_suited/outdata"
dsxrf = DataSXRF()

In [12]:
%%time
synt_data = dsxrf.read(datadir)

Reading SXRF data from /home/rosario/xmimsim/syntetic/ElGreco_suited/outdata
CPU times: user 368 ms, sys: 102 ms, total: 470 ms
Wall time: 25.4 s


In [13]:
for spe in synt_data.spe_objs:
    print(spe.reflayer_atomic_num, spe.weight_fractions)
    for lines in spe.fluorescence_lines:
        print(lines.symbol, lines.lines)

[24, 25] [94.62, 5.38]
Ar {'K': 20.456827999999998, 'L': 0, 'others': 0}
Cr {'K': 407.43800000000005, 'L': 0, 'others': 0}
Mn {'K': 23.86969, 'L': 0, 'others': 0}
Pb {'K': 0, 'L': 3784.1218183184997, 'others': 160.81804592361402}
[24, 25] [94.49, 5.51]
Ar {'K': 20.443703, 'L': 0, 'others': 0}
Cr {'K': 232.47789, 'L': 0, 'others': 0}
Mn {'K': 13.928266999999998, 'L': 0, 'others': 0}
Pb {'K': 0, 'L': 3771.4261563592986, 'others': 170.64617783791496}
[24, 25] [92.42, 7.58]
Ar {'K': 20.441208000000003, 'L': 0, 'others': 0}
Cr {'K': 145.10903999999996, 'L': 0, 'others': 0}
Mn {'K': 12.194958000000002, 'L': 0, 'others': 0}
Pb {'K': 0, 'L': 3785.1093143345006, 'others': 176.51206331334902}
[24, 25] [98.65, 1.35]
Ar {'K': 20.413238, 'L': 0, 'others': 0}
Cr {'K': 395.271, 'L': 0, 'others': 0}
Mn {'K': 5.5896609999999995, 'L': 0, 'others': 0}
Pb {'K': 0, 'L': 3783.6745176031004, 'others': 162.76454482445297}
[24, 25] [94.2, 5.8]
Ar {'K': 20.274627000000002, 'L': 0, 'others': 0}
Cr {'K': 172.7213

Ti {'K': 572.594, 'L': 0, 'others': 0}
Ba {'K': 0, 'L': 15.195841480888102, 'others': 5.861178013256401e-07}
Pb {'K': 0, 'L': 1511.3052805167, 'others': 144.71527941705702}
[22, 56] [90.14, 9.86]
Ar {'K': 20.508199, 'L': 0, 'others': 0}
Ti {'K': 296.148, 'L': 0, 'others': 0}
Ba {'K': 0, 'L': 18.8634817590112, 'others': 8.60563586958598e-07}
Pb {'K': 0, 'L': 2392.2511030288993, 'others': 162.18173839528004}
[22, 56] [82.61, 17.39]
Ar {'K': 20.68178, 'L': 0, 'others': 0}
Ti {'K': 372.3108, 'L': 0, 'others': 0}
Ba {'K': 0, 'L': 45.66636998426531, 'others': 1.8660025772610728e-06}
Pb {'K': 0, 'L': 3696.0042161291, 'others': 152.62783217276703}
[22, 56] [89.49, 10.51]
Ar {'K': 20.639846, 'L': 0, 'others': 0}
Ti {'K': 131.27499, 'L': 0, 'others': 0}
Ba {'K': 0, 'L': 8.9929946331998, 'others': 4.6792305084846403e-07}
Pb {'K': 0, 'L': 3902.4804344746, 'others': 175.18389766513903}
[22, 56] [91.44, 8.56]
Ar {'K': 20.648797, 'L': 0, 'others': 0}
Ti {'K': 402.8928000000001, 'L': 0, 'others': 0}
B

Fe {'K': 303.32910000000004, 'L': 0, 'others': 0}
Co {'K': 27.14305, 'L': 0, 'others': 0}
Pb {'K': 0, 'L': 2964.5528146018005, 'others': 163.323361856039}
[26, 27] [96.13, 3.87]
Ar {'K': 20.602187, 'L': 0, 'others': 0}
Fe {'K': 586.5509999999999, 'L': 0, 'others': 0}
Co {'K': 22.918492, 'L': 0, 'others': 0}
Pb {'K': 0, 'L': 3592.176033243001, 'others': 146.276137372634}
[26, 27] [88.76, 11.24]
Ar {'K': 20.336115000000003, 'L': 0, 'others': 0}
Fe {'K': 547.0374999999999, 'L': 0, 'others': 0}
Co {'K': 67.32748000000001, 'L': 0, 'others': 0}
Pb {'K': 0, 'L': 3702.3182566821997, 'others': 145.65163777507098}
[26, 27] [91.66, 8.34]
Ar {'K': 20.511952, 'L': 0, 'others': 0}
Fe {'K': 35.62827, 'L': 0, 'others': 0}
Co {'K': 3.1660510000000004, 'L': 0, 'others': 0}
Pb {'K': 0, 'L': 3806.5142852475997, 'others': 182.87035005830595}
[26, 27] [85.55, 14.45]
Ar {'K': 20.615383, 'L': 0, 'others': 0}
Fe {'K': 254.89810000000003, 'L': 0, 'others': 0}
Co {'K': 41.789500000000004, 'L': 0, 'others': 0}
Pb

In [14]:
dir(synt_data)

['_Data__resample_x',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__read__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_get_labels',
 'calibrate_from_file',
 'f_resample_y',
 'fce_calibration',
 'get_data_and_labels',
 'load_h5',
 'metadata',
 'name',
 'process_file',
 'read',
 'read_params',
 'resample',
 'save_h5',
 'shape',
 'spe_objs',
 'x']

In [15]:
dir(synt_data.spe_objs[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'channel',
 'counts',
 'energy',
 'fluorescence_lines',
 'from_array',
 'from_file',
 'get_fluorescence_lines',
 'get_metadata',
 'reflayer_atomic_num',
 'reflayer_thickness',
 'sublayer_thickness',
 'weight_fractions']

In [16]:
import numpy as np
symbols = ["Ca", "Co", "Cu", "Fe", "Hg", "K", "Mn", "Pb", "S", "Sn"]
lines = ["K", "K", "K", "K", "L", "K", "K", "L", "K", "L"]

In [17]:
%%time
synt_data = synt_data.get_data_and_labels(symbols, lines)

CPU times: user 7.32 ms, sys: 7.14 ms, total: 14.5 ms
Wall time: 13.4 ms


In [18]:
print(f"data shape: {synt_data.data.shape}")
print(f"labels shape: {synt_data.labels.shape}")
print(synt_data.labels)

data shape: (2189, 2048)
labels shape: (2189, 10)
[[   0.            0.            0.         ... 3784.12181832
     0.            0.        ]
 [   0.            0.            0.         ... 3771.42615636
     0.            0.        ]
 [   0.            0.            0.         ... 3785.10931433
     0.            0.        ]
 ...
 [   0.            0.            0.         ... 3918.9094597
     0.            0.        ]
 [   0.            0.            0.         ... 3376.31509631
     0.            0.        ]
 [   0.            0.            0.         ... 3812.98460689
     0.            0.        ]]
